# Adding puntuation algorythms - NLP project
### Jan Wasilewski

In [2]:
#The most basic algorythm
def addPunctuationBasic(string):
    return string.capitalize() + '.'

#Example
addPunctuationBasic('i think therefore i am')

'I think therefore i am.'

In [3]:
chars = ['.',',',';',':','?','!']

def token(string):
    for char in chars:
        string = string.replace(char, ' ' + char)
    token_string = string.split()
    return token_string

def verifyPuntuation(string_check, string_test):
    errors = []
    j = 0
    i = 0
    token_check = token(string_check)
    token_test = token(string_test)
    while i < len(token_test) and j < len(token_check) :
        if token_test[i] == token_check[j]:
            i+=1
            j+=1
        elif token_test[i] in chars and not token_check[j] in chars:
            errors.append(('I', i))
            i+=1
        elif not token_test[i] in chars and token_check[j] in chars:
            errors.append(('D', i))
            j+=1
        else:
            errors.append(('S', i))
            i+=1
            j+=1 
    return errors

# Example from email:
verifyPuntuation("Hello. What's your name?", "Hello, what's Your, name?")

[('S', 1), ('S', 2), ('S', 3), ('I', 4)]

# Errors analysis

In [4]:
import pandas as pd
import numpy as np
import os

os.chdir("/home/jan/Documents/Uni/erasmus/PLN-MULCIA-Junio-2020-Dataset")


strings_plain = pd.read_csv("PunctuationTask_test.txt", header=None)

strings_check = pd.read_csv("PunctuationTask_check.txt", header=None, sep = "/n")

strings_plain = np.reshape(strings_plain.values, -1)
strings_check = np.reshape(strings_check.values, -1)

/home/jan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [5]:
from sklearn.metrics import classification_report


def getPuntuation(string):
    split = string.split()
    result = np.full(len(split), ' ')
    for (index, word) in enumerate(split):
        for char in chars:
            if char in word:
                result[index] = char
                break
    return result

def verifyPuntuationMetrics(strings_plain, strings_check, model):
    strings_predict = [model(string_plain) for string_plain in strings_plain]
    pred = np.array([getPuntuation(string) for string in strings_predict])
    original = np.array([getPuntuation(string) for string in strings_check])
    index = np.where(np.array([len(v) for v in pred]) == np.array([len(v) for v in original]))[0]
    pred_trunc = pred[index]
    original_trunc = original[index]
    return classification_report(np.concatenate(original_trunc), np.concatenate(pred_trunc))

In [6]:
def ModelBasicMetrices(strings_plain, strings_check):
    return verifyPuntuationMetrics(strings_plain, strings_check, addPunctuationBasic)

In [88]:
# Basic model metrices
Basic = ModelBasicMetrices(strings_plain, strings_check)
print(Basic)

              precision    recall  f1-score   support

                   0.89      1.00      0.94      1340
           !       0.00      0.00      0.00         2
           ,       0.00      0.00      0.00        95
           .       0.97      0.54      0.69       106
           :       0.00      0.00      0.00        10
           ?       0.00      0.00      0.00         4

    accuracy                           0.90      1557
   macro avg       0.31      0.26      0.27      1557
weighted avg       0.84      0.90      0.86      1557



# Gram model

In [20]:
# Train strings upload
File_train = open("PunctuationTask_train.txt","r")

strings_train = File_train.read().replace('\n', '')
strings_train_tokens = np.array(token(strings_train))
N = len(strings_train_tokens)
N

2454275

In [91]:
def check_gram(token1, token2, token3):
    inds = [i+3 for i in range(N-3) if (strings_train_tokens[i:i+3] == [token1, token2, token3]).all()]
    stats = []
    len_inds = len(inds) 
    if len_inds > 0:
        for char in chars:
            stats.append(list(strings_train_tokens[inds]).count(char))
        if len_inds - sum(stats) > max(stats):
            recomendation = ''
        else:
            recomendation = chars[np.argmax(stats)]
    else:
        recomendation = ''
    return recomendation

def untocken(tockens):
    string = tockens[0]
    for tocken in tockens[1:]:
        if tocken in chars:
            string = string + tocken
        else:
            string = string + ' ' + tocken
    return string

def addPunctuationGram(string):
    token_string = token(string)
    i = 0
    while i < len(token_string) - 2:
        recomendation = check_gram(token_string[i], token_string[i+1], token_string[i+2])
        if len(recomendation) > 0:
            token_string.insert(i+3, recomendation)
        i += 1
    untockened_string = untocken(token_string).capitalize()
    if not untockened_string.endswith('.'):
        untockened_string = untockened_string + '.'
    return untockened_string

In [92]:
# example
addPunctuationGram('it can be a very complicated thing the ocean')

'It can be a very complicated thing, the ocean.'

In [84]:
def ModelGramMetrices(strings_plain, strings_check):
    return verifyPuntuationMetrics(strings_plain, strings_check, addPunctuationGram)

In [89]:
# 4-gram model metrices
Gram = ModelGramMetrices(strings_plain[:5], strings_check[:5])
print(Gram)

              precision    recall  f1-score   support

                   0.96      0.99      0.98        82
           ,       0.50      0.14      0.22         7
           .       0.44      1.00      0.62         4
           ;       0.00      0.00      0.00         1
           ?       0.00      0.00      0.00         1

    accuracy                           0.91        95
   macro avg       0.38      0.43      0.36        95
weighted avg       0.89      0.91      0.88        95

